# Projeto: Análise de Sentimentos em Discursos Políticos

<font color='gray'>Parte 1: Coleta de dados</font><br>
<font color='black'>**Parte 2: Limpeza de dados**</font><br>
<font color='gray'>Parte 3: Modelagem e Análise de Sentimentos</font>

![](./images/data-cleaning-1.svg)

Esta é a segunda parte deste projeto, no qual faremos toda limpeza e extração dos dados do código HTML para deixar no formato ideal para análises, que ficará para o próximo *notebook*.

## Importando as bibliotecas

Iniciaremos importando as bibliotecas que serão utilizadas. As principais delas são:
* **pickle:** para importar os dados salvos no notebook anterior e também para salvar após limpeza.
* **re:** para expressões regulares para extrair padrões.

Além das outras básicas para manipulação dos dados.

In [1]:
# Imports

# Manipulação de dados
import os
import re
import sys
import time
import string
import pickle
import requests
import datetime
import numpy as np
import pandas as pd

# Definimos o parâmetro abaixo para evitar o erro:
# RecursionError: maximum recursion depth exceeded
# Ao gravar os resultados em disco
sys.setrecursionlimit(10000)

## Limpeza de Dados

Iniciaremos importando os dados coletados anteriormente, comos os links e os discursos utilizando a biblioteca `pickle`.

In [2]:
# Abrindo o banco de dados
with open('dados/db_links.pickle', 'rb') as read_file:
    db_links = pickle.load(read_file)
    
with open('dados/db_discursos.pickle', 'rb') as read_file:
    db_discursos = pickle.load(read_file)

Os dois objetos que criamos estão em formato de lista. Vamos dar uma olhada nos elementos de cada um.

In [3]:
# visualizando os dois primeiros elementos da lista de links.
db_links[0:2]

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-no-lancamento-dos-programas-codex-e-super-br-e-8o-revogacao-palacio-do-planalto',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-na-cupula-do-g20-videoconferencia-palacio-do-planalto-1']

In [4]:
# visualizando o primeiro elemento da lista dos discursos
db_discursos[0:1]

[<html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths js-focus-visible" data-js-focus-visible="" lang="pt-br" xml:lang="pt-br" xmlns="http://www.w3.org/1999/xhtml"><head><style type="text/css">.tippy-touch{cursor:pointer!important}.tippy-notransition{transition:none!important}.tippy-popper{max-width:350px;-webkit-perspective:700px;perspective:700px;z-index:9999;outline:0;transition-timing-function:cubic-bezier(.165,.84,.44,1);pointer-events:none;line-height:1.4}.tippy-popper[data-html]{max-width:96%;max-width:calc(100% - 20px)}.tippy-popper[x-placement^=top] .tippy-backdrop{border

Já analisamos a página no site e notamos que alguns discursos iniciam com a data e outros não, mas como a data e a hora são informações importantes para nossas análises, encontramos outro campo dentro do próprio código para extrairmos, as *tags* e *classes* correspondentes e vamos criar uma função para isso.

In [5]:
# criando função para extrair data e hora
def extrair_data_hora(source_object):
    
    # lista vazia para receber o resultado
    date_hour = []
    
    # loop para extração das informações
    for s in source_object:
        date = s.find_all('span', {'class': 'value'})[0].text
        date_clean = pd.to_datetime(date)
        date_hour.append(date_clean)
    
    # criando um dataframe
    df = pd.DataFrame(date_hour)
    df.columns = ['data_hora']
    df['data'] = df['data_hora'].dt.date
    df['hora'] = df['data_hora'].dt.time
    df = df.drop('data_hora', axis=1)
    
    # retornando o resultado
    return df

In [6]:
# executando a função colocando em um objeto
date_hour = extrair_data_hora(db_discursos)

# visualizando as primeiras linhas
date_hour.head()

,data,hora
0,2020-11-26,19:33:00
1,2020-11-22,11:16:00
2,2020-11-21,11:20:00
3,2020-11-18,14:52:00
4,2020-11-17,18:12:00


Agora vamos extrair as informações principais para as análises que são os discursos. Já analisamos a página e vimos que ficam na *tag* "p".

In [7]:
# função para extrair o texto
def extrai_texto(source_object):
    
    # lista vazia para armazenar os textos
    text_list = []
    
    # loop para extração
    for s in source_object:
        text = s.find_all('p')
        text_list.append(text)
    
    # retornando o resultado
    return text_list

In [ ]:
# # lista os tamanhos dos textos
# discurso_lenght = []

# # loop para extração do tamanho
# for i in extrai_texto(db_discursos):
#     discurso_lenght.append(len(i))

In [ ]:
# # converte para array numpy
# discurso_lenght_array = np.array(discurso_lenght)

# # visualizando o resultado
# discurso_lenght_array

Executando a função e armazenando o resultado em um objeto.

In [8]:
# Extrai os discursos
primeiros_discursos = extrai_texto(db_discursos)

# checando o primeiro texto/elemento da lista
primeiros_discursos[0]

[<p><span><span> </span></span></p>,
 <p dir="ltr"><span><span class="Apple-tab-span"> </span></span><span>Pode ficar tranquilo, Pedro Guimarães, que o nosso Posto Ipiranga é insubstituível, então não falarei de economia. Todos nós temos momentos difíceis de preocupações, de decisões que, por vezes, não agrada muita gente, mas hoje é um dia muito especial, só tive boas notícias até agora.</span>Primeiro, o que nós acabamos de assistir, havia assim lá atrás conversado com o Jorge da possibilidade que cada vez que nós tivéssemos um ato, um decreto, uma portaria,  tivéssemos revogado duas outras, e ele aqui cumpriu essa missão com galhardia juntamente com os seus  assessores, muito obrigado Jorge! Confesso que vamos sentir saudades de vocês.</p>,
 <p dir="ltr">E mais ainda ao longo desses dois anos de mandato nós revogamos mais atos do que outros presidentes o fizeram em vinte anos. Então a questão da desburocratização, desregulamentação, é muito bem-vinda, e depois um aplicativo chamado 

In [ ]:
# primeiros_discursos[29]

In [ ]:
# # Estes são os índices que precisam ser re-extraídos com get_text2
# # Isso é necessário pois as páginas tem diferenças entre elas
# indices = list(np.argwhere(discurso_lenght_array == 1).flatten())

In [ ]:
# indices

In [ ]:
# # função para nova extração do texto
# def extrai_texto2(source_object):
#     text_list = []
#     for s in source_object:
#         text = s.find('p').parent
#         text_list.append(text)
#     return text_list

In [ ]:
# # loop para extrair textos mais claros
# melhores_discursos = []
# for i in indices:
#     melhores_discursos.append(extrai_texto2(db_discursos[i]))

In [ ]:
# melhores_discursos

In [ ]:
# # depois de re-extrair, se necessários a gente substitui os anteriores
# for (indices, melhores_discursos) in zip(indices, melhores_discursos):
#     primeiros_discursos[indices] = melhores_discursos

In [ ]:
# primeiros_discursos[29]

Está quase lá, vamos usar o *regex* para extrair os caracteres indesejados. Criaremos duas funções e quando executarmos o resultado será os textos limpos, armazenados em um dataframe junto com os links. Notamos que quando o texto pula uma linha, no código coletado ele fica com duas vírgulas, temos que removê-las.

In [9]:
# função para remover tags html
def remove_html_tags(text):
    
    # remove as tags
    clean = re.compile('<.*?>')
    
    # remova mais as vírgulas e espaços duplos.
    clean1 =  re.sub(clean, '', text).replace(',', '').replace('  ', ' ')
    
    # retorna o resultado
    return re.sub('\s+', ' ', clean1)

# função para limpeza de dados
def limpa_dados(transcript_list, link_list):
    
    # lista para receber o texto limpo
    text = []
    
    # loop executando a função acima
    for i in transcript_list:
        cleaned = remove_html_tags(str(i))
        text.append(cleaned)
        
    # armazenando em um dataframe
    df = pd.DataFrame([link_list, text]).T
    
    # renomeando as colunas
    df.columns = ['link', 'text']
    
    # retornando o resultado
    return df

In [10]:
# executando a função para limpeza
dados_limpos = limpa_dados(primeiros_discursos, db_links)

# visualiszando as primeiras linhas
dados_limpos.head()

,link,text
0,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[ Pode ficar tranquilo Pedro Guimarães que o n...
1,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Senhoras e Senhores Primeiramente parabenizo ...
2,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Senhoras e Senhores Antes de adentrarmos o te...
3,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Mais que emocionado eu estou muito feliz em p...
4,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Brasília/DF 17 de novembro de 2020 Sua Excelê...


Por fim precisamos remover alguns caracteres que ficaram nos textos, também utilizaremos o *regex* para isso, armazenaremos em uma nova coluna do dataframe e concatenamos os dataframe de data e hora.

In [11]:
# padrão para remover
toremove = "'\\,\"\[\(\]\)-–"

# funções para extrair caracteres e padrões indesejados dos dados
punc_lower = lambda x: re.sub('[%s]' % re.escape(toremove), '', x.lower())
remove_space = lambda x: x.replace('  ', ' ')

# executando a função e armazenando em uma outra coluna no df
dados_limpos['monologue'] = dados_limpos['text'].map(punc_lower).str.strip()

# concatenando o df de data e hora
df_dados_limpos = pd.concat([date_hour, dados_limpos], axis=1)

Vamos checar o resultado final.

In [12]:
df_dados_limpos[0:4]

,data,hora,link,text,monologue
0,2020-11-26,19:33:00,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[ Pode ficar tranquilo Pedro Guimarães que o n...,pode ficar tranquilo pedro guimarães que o nos...
1,2020-11-22,11:16:00,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Senhoras e Senhores Primeiramente parabenizo ...,senhoras e senhores primeiramente parabenizo a...
2,2020-11-21,11:20:00,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Senhoras e Senhores Antes de adentrarmos o te...,senhoras e senhores antes de adentrarmos o tem...
3,2020-11-18,14:52:00,https://www.gov.br/planalto/pt-br/acompanhe-o-...,[Mais que emocionado eu estou muito feliz em p...,mais que emocionado eu estou muito feliz em po...


Vamos salvar os dados como já fizemos anteriormente e se for necessário podemos coletar mais dados, mas a princípio ficaremos com esses.

In [13]:
# salvamos os discusos agora limpos
with open('dados/db_discurso_limpo.pickle', 'wb') as to_write:
    pickle.dump(df_dados_limpos, to_write)

Também vou salvar os dados em excel para checar se não precisa de mais alguma limpeza, no excel pode ser melhor pra checar essa informação.

In [14]:
# Criando um objeto com a engine xlswriter
writer = pd.ExcelWriter('dados/db_discurso_limpo.xlsx', engine='xlsxwriter')

# Convertendo o dataframe para excel
df_dados_limpos.to_excel(writer, sheet_name='Sheet1', index=False)

# fechando a instancia do pandas excel
writer.save()